# Getting started

This is a short introduction to sinaps for new users.

## Overview

The primary data structures of sinaps are:

* `Section` wich aims to represent a segment of a neuron with uniform physicals values
* `Neuron` wich aims to represent a complete neuron is a directed graph whose edges are of `Section` type.
* `Channels` wich aim to represent ionic channels with various dynamics. Several channels are already implemented, such as Hodgkin-Huxley channels, Heavyside currents or AMPA receptors. <!-- Exhaustive list can be found in section XXX -->

The outputs of a simulation are:

* Electric potential for each time step and position in the neuron
* Concentration of species for each time step and position in the neuron



The package can be imported as follows:

In [ ]:
import sinaps as sn

### Object creation
Creating an empty `Neuron`:

In [ ]:
nrn = sn.Neuron()

Creating a `Section`, letting sinaps setting default attribute :

In [ ]:
sec = sn.Section()

Adding a `HodgkinHuxley` [channel](https://en.wikipedia.org/wiki/Hodgkin%E2%80%93Huxley_model) to the newly created section:

In [ ]:
sec.add_channel(sn.channels.Hodgkin_Huxley())

Adding a Hodgkin-Huxley type calcium channel:

In [ ]:
sec.add_channel(sn.channels.Hodgkin_Huxley_Ca())


Adding a `HeavysideCurrent` channel with a current of `100` pA from `2` to `4` ms at the beginning of the section:

In [ ]:
sec.add_channel(sn.channels.HeavysideCurrent(100,2,4),0)

Adding the section `s` to the neuron `n` as an edge between nodes `0` and `1` :

In [ ]:
nrn.add_section(sec,0,1)

To add two new sections sec 2 with a radius of 2 um and sec3 with a length of 200 um:

In [ ]:
sec2 = sn.Section(a = 2)
sec3 = sn.Section(L = 200)
nrn.add_sections_from_dict({
    sec2:(1,2),
    sec3:(1,3)
})
    

Plotting the neuron structure:

In [ ]:
nrn.plot()

Adding calcium ions in the model:

In [ ]:
nrn.add_species(sn.Species.Ca,C0=2E-4, D=100) # For the sake of the example, we increase the calcium diffusion coefficient to speed-up its dynamics, in order to observe variations within 50 ms. 

### Simulation running

Creating a `Simulation` of neuron `n` with spatial resolution `10` um:

In [ ]:
sim = sn.Simulation(nrn,dx=10)

Running the simulation for timespan `0` to `20` ms:

In [ ]:
sim.run((0,50))

Results of the simulation are stored as a [pandas](https://pandas.pydata.org/) Dataframe:

### Viewing results

Plotting up to 10 curves distributed evenly on the neuron:

In [ ]:
sim.plot()

Plotting the Hodgkin-Huxley currents:

In [ ]:
sim.plot.I(sn.channels.Hodgkin_Huxley)

Getting a field view of the potential:

In [ ]:
sim.plot.V_field()

Running the electrodiffusion part:

In [ ]:
sim.run_diff(max_step=1)

Plotting the clacium concentration dynamics

In [ ]:
sim.plot.C(sn.Species.Ca)

In [ ]:
sim.plot.C_field(sn.Species.Ca)

## Section

The class `Section` represents a section of neuron with uniform physical values

The characteristics of a section are :

* The length `L` in **μm**
* The radius `a` in **μm**
* The menbrane capacitance `C_m` in **μF/cm²**
* The longitunal resistance `R_l` in **Ohm.cm**
* The initial potential `V0` in **mV**

### Default parameters

In [ ]:
sec0 = sn.Section(name="Sample section 1")
sec0

### Customized values

Sinaps uses the [param](https://param.holoviz.org/) library. You can set custom parameter at the object creation :

In [ ]:
sec1 = sn.Section(L=50,a=2,name="Sample section 2")
sec1

You can also set the attribute value once the object created :

In [ ]:
sec1.R_l=100
sec1

## Channels

Ion channels can be added to a section.

There are two types of channels
+ Point channels
+ Density channels




### Density Channels
Density channels are used to model channels that are distributed everywhere on a section. The current is given per unit of membrane surface

#### Leak Channel


In [ ]:
lc1=sn.channels.LeakChannel()
lc1

In [ ]:
lc2=sn.channels.LeakChannel(
            Veq=10, #mV
            G_m= 1 #mS/cm²
            )
lc2

In [ ]:
sec0.add_channel(lc1)
sec0

### Point Channels
Point channels are used to model channel in specific location of the section, the given current is in pA (not relative to the section membrane surface).

### Constant current

In [ ]:
pc=sn.channels.ConstantCurrent(1)

In [ ]:
sec0.add_channel(pc,0)
sec0

## Neuron

The class `Neuron` represents a set of sections connected together

In [ ]:
nrn2=sn.Neuron()

In [ ]:
nrn2.add_section(sec0,0,1)
nrn2.add_section(sec1,1,2)

The structure of the neuron is stored in the attribute `sections` wich is a Dict with the section as key and the nodes connected by the section as values (2-tuple) :

In [ ]:
nrn2.sections

### Accessing the sections
By index in the neuron structure

In [ ]:
nrn2[0]

By name

In [ ]:
nrn2['Sample section 2']

Not that if sections have same names or same part of the name a list of section will be returned

In [ ]:
nrn2['Sample section']

You can change the parameters of multiples section at once :

In [ ]:
nrn2['Sample section'].C_m=1.5

Access all the sections :

In [ ]:
nrn2[:]

In [ ]:
nrn2.plot()

## Simulation

The class simulation is linked to a specific neuron and is used to run voltage propagation simulation and electrodiffusion simulations with some custom spatial and time resolution.
The object stores also results of the simulation

Create the simulation with neuron `nrn` and spatial resolution of `20 um` :

In [ ]:
sim=sn.Simulation(nrn,dx=20)

Run the simulation for timespan `0 - 300 ms` : 

In [ ]:
sim.run((0,300))

Acces the results of the simulation, potential for each time and position :

In [ ]:
sim.V

In [ ]:
sim['Sample section 2'].plot()

In [ ]:
sim[:].plot()

## Species

The species class 

Defining a new species:

In [ ]:
nrn2.add_species(sn.Species.Ca,C0=2E-4, D=100)